In [26]:
# IMPORT LIBRARY 

from __future__ import print_function, division
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import matplotlib.patches as mpatches
import seaborn as sns

from sklearn.model_selection import train_test_split

# 随机森林, GBR, KNN, LR, Decision Tree, GNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.utils import check_random_state
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
warnings.filterwarnings('ignore')
rs = check_random_state(4421)

In [27]:
# 数据加载和转换
# 数据读取: 

data = pd.read_csv('games.csv')

In [28]:
data

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,...,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,...,5,0,0,1,1,114,67,43,16,51
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,...,2,0,0,0,0,11,67,238,51,420
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,...,2,0,0,1,0,157,238,121,57,28
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,...,0,0,0,0,0,164,18,141,40,51
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,...,3,0,0,1,0,86,11,201,122,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51485,3308904636,1503076540231,1944,9,2,1,2,2,0,2,...,10,2,0,4,0,55,-1,90,238,157
51486,3215685759,1496957179355,3304,9,2,1,1,2,2,2,...,11,7,4,4,1,157,55,119,154,105
51487,3322765040,1504029863961,2156,9,2,2,2,2,0,1,...,10,2,0,2,0,113,122,53,11,157
51488,3256675373,1499562036246,1475,9,2,2,2,2,0,2,...,11,3,0,1,0,154,39,51,90,114


In [29]:
data.drop(["t1_champ1id","t1_champ2id",
                      "t1_champ3id","t1_champ4id","t1_champ5id","t2_champ1id",
                      "t2_champ2id","t2_champ3id","t2_champ4id","t2_champ5id","t1_ban1","t1_ban2","t1_ban3",
                      "t1_ban4","t1_ban5","t2_ban1","t2_ban2","t2_ban3","t2_ban4","t2_ban5","seasonId"],axis = 1,inplace = True)

In [30]:
Team1 = data[['gameDuration','firstBlood','firstTower','firstInhibitor','firstBaron','firstDragon','firstRiftHerald','t1_towerKills','t1_inhibitorKills','t1_baronKills','t1_dragonKills','t1_riftHeraldKills','winner']]
Team2 = data[['gameDuration','firstBlood','firstTower','firstInhibitor','firstBaron','firstDragon','firstRiftHerald','t2_towerKills','t2_inhibitorKills','t2_baronKills','t2_dragonKills','t2_riftHeraldKills','winner']]
teams= pd.concat([Team1,Team2],axis=0)

In [31]:
data

,gameId,creationTime,gameDuration,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,...,t2_champ3_sum2,t2_champ4_sum1,t2_champ4_sum2,t2_champ5_sum1,t2_champ5_sum2,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills
0,3326086514,1504279457970,1949,1,2,1,1,1,1,2,...,4,14,4,4,3,5,0,0,1,1
1,3229566029,1497848803862,1851,1,1,1,1,0,1,1,...,4,4,14,4,7,2,0,0,0,0
2,3327363504,1504360103310,1493,1,2,1,1,1,2,0,...,12,4,11,7,4,2,0,0,1,0
3,3326856598,1504348503996,1758,1,1,1,1,1,1,0,...,3,4,14,4,7,0,0,0,0,0
4,3330080762,1504554410899,2094,1,2,1,1,1,1,0,...,4,4,12,4,7,3,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51485,3308904636,1503076540231,1944,2,1,2,2,0,2,0,...,7,3,4,11,4,10,2,0,4,0
51486,3215685759,1496957179355,3304,2,1,1,2,2,2,2,...,4,4,12,14,4,11,7,4,4,1
51487,3322765040,1504029863961,2156,2,2,2,2,0,1,0,...,4,12,4,14,4,10,2,0,2,0
51488,3256675373,1499562036246,1475,2,2,2,2,0,2,0,...,14,12,4,11,4,11,3,0,1,0


In [36]:
data=data[["winner","firstBlood","firstTower","firstInhibitor","firstBaron","firstDragon","firstRiftHerald","t1_towerKills","t1_inhibitorKills",'t1_baronKills',
       't1_dragonKills','t2_towerKills','t2_inhibitorKills', 't2_baronKills', 't2_dragonKills'
       ]]

In [37]:
X = data.drop(['winner'], axis=1)
Y = data['winner'].values

In [38]:
# Decision Tree
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.2,random_state=rs)

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [39]:
criterion=["gini","entropy"]
max_depth=range(1,10,3)
splitter=["best","random"]
dt=DecisionTreeClassifier()
grid_decision_tree=GridSearchCV(estimator=dt,param_grid=dict(criterion=criterion,max_depth=max_depth,splitter=splitter))

In [40]:
grid_decision_tree.fit(X_train,y_train)
print("best score: ", grid_decision_tree.best_score_)
print("best param: ", grid_decision_tree.best_params_)

best score:  0.9700184883853907
best param:  {'criterion': 'gini', 'max_depth': 7, 'splitter': 'best'}


In [42]:
dt2=DecisionTreeClassifier(criterion="gini",max_depth=7, splitter="best")
dt2.fit(X_train,y_train)
print("score:", dt2.score(X_test,y_test))

score: 0.964362012041173


In [43]:
scenario={"feature":["first_blood","first_tower","first_inhibitor","first_Baron","first_Dragon","first_RiftHerald",
"t1_tower","t1_inhibitor","t1_baron","t1_dragon","t2_tower","t2_inhibitor","t2_baron","t2_dragon"],
         "value":[1,1,2,1,1,1,10,2,1,4,7,2,1,1]}
scen=pd.DataFrame(scenario)
scen.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
feature,first_blood,first_tower,first_inhibitor,first_Baron,first_Dragon,first_RiftHerald,t1_tower,t1_inhibitor,t1_baron,t1_dragon,t2_tower,t2_inhibitor,t2_baron,t2_dragon
value,1,1,2,1,1,1,10,2,1,4,7,2,1,1


In [44]:
x1=[[1,1,2,1,1,1,10,2,1,4,7,2,1,1]]
c=dt2.predict_proba(x1).reshape(-1,1)
print("winner is :" , dt2.predict(x1) )
print("first team win probability is % ", list(c[0]*100),"\nsecond team win probability is %:",list(c[1]*100)  )


winner is : [1]
first team win probability is %  [79.88165680473372] 
second team win probability is %: [20.118343195266274]


In [45]:
x1=[[0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
c=dt2.predict_proba(x1).reshape(-1,1)
print("winner is :" , dt2.predict(x1) )
print("first team win probability is % ", list(c[0]*100),"\nsecond team win probability is %:",list(c[1]*100)  )


winner is : [1]
first team win probability is %  [53.964757709251096] 
second team win probability is %: [46.035242290748904]
